In [1]:
import os
%pwd

'd:\\Machine_Learning\\Fetal-Health-Classification\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\Machine_Learning\\Fetal-Health-Classification'

In [3]:
# Data Ingestion Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from FetalHealthC.utils.common import read_yaml, create_directories
from FetalHealthC.constants import *

In [9]:
# Configuration Manager for DataIngestion
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH
            ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig: 
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [10]:
from FetalHealthC.logging import logger
import urllib.request as request
import zipfile

In [11]:
class DataIngestion:
    def __init__(
            self,
            data_ingestion_config: DataIngestionConfig
            ):
        self.config = data_ingestion_config
    
    def download_file(self):
        
        if os.path.exists(self.config.local_data_file):
            logger.info(f"File already exists: {self.config.local_data_file}")
            
        else :
            filename, headers = request.urlretrieve(
                url= self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"File downloaded: {filename} with the following \n {headers}")

    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)  

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-01-10 20:18:00,158: INFO: common: yaml file config\config.yaml loaded successfully]
[2024-01-10 20:18:00,164: INFO: common: yaml file params.yaml loaded successfully]
[2024-01-10 20:18:00,168: INFO: common: created directory at: artifacts]
[2024-01-10 20:18:00,173: INFO: common: created directory at: artifacts/data_ingestion]


[2024-01-10 20:18:02,047: INFO: 1479116766: File downloaded: artifacts/data_ingestion/data.zip with the following 
 Connection: close
Content-Length: 46932
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c7e9d2adfee41fb833ae5a2dd946af024529d84bce655589c561d070059ca9fc"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8CA6:7817:8F9D93:AE2971:659EAE20
Accept-Ranges: bytes
Date: Wed, 10 Jan 2024 14:48:01 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10246-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1704898081.210578,VS0,VE745
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 4a0852657b4e276749c4c58d9aea76e553833c62
Expires: Wed, 10 Jan 2024 14:53:01 GMT
Source-Age: 0

]
